### A crossfunctional team with more than 1 member of datascientist working on ML model

##### MLflow setup

1. tracking server: yes, local server
2. backend store : sqlite database
3. artifacts store: local  filesystem

-- to run the server locally, you need to run the code on the terminal: 

mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI:'{mlflow.get_tracking_uri()}'")

tracking URI:'http://127.0.0.1:5000'


In [4]:
mlflow.list_experiments()

/tmp/ipykernel_1846/546495964.py:1: FutureWarning: ``mlflow.tracking.fluent.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  mlflow.list_experiments()
/home/bafna0109/anaconda3/envs/deepl/lib/python3.11/site-packages/mlflow/tracking/fluent.py:1065: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  return MlflowClient().list_experiments(


[<Experiment: artifact_location='./artifacts_local/0', creation_time=1667817669338, experiment_id='0', last_update_time=1667817669338, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [6]:
mlflow.set_experiment("my_experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C":0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X,y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y,y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2022/11/07 10:45:18 INFO mlflow.tracking.fluent: Experiment with name 'my_experiment-1' does not exist. Creating a new experiment.


default artifacts URI: './artifacts_local/1/0226e722f00a4413af31557fe1d6431a/artifacts'


/home/bafna0109/anaconda3/envs/deepl/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
mlflow.list_experiments()

/tmp/ipykernel_1846/546495964.py:1: FutureWarning: ``mlflow.tracking.fluent.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  mlflow.list_experiments()
/home/bafna0109/anaconda3/envs/deepl/lib/python3.11/site-packages/mlflow/tracking/fluent.py:1065: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_experiments`` is deprecated. This method will be removed in a future release. Use ``search_experiments()`` instead.
  return MlflowClient().list_experiments(


[<Experiment: artifact_location='./artifacts_local/0', creation_time=1667817669338, experiment_id='0', last_update_time=1667817669338, lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', creation_time=1667817918642, experiment_id='1', last_update_time=1667817918642, lifecycle_stage='active', name='my_experiment-1', tags={}>]

#### registering models

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient('http://127.0.0.1:5000')

In [10]:
client.search_registered_models()

[]

In [11]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

/tmp/ipykernel_1846/2480069845.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_run_infos`` is deprecated. This method will be removed in a future release. Use ``search_runs()`` instead.
  run_id = client.list_run_infos(experiment_id='1')[0].run_id
Successfully registered model 'iris-classifier'.
2022/11/07 10:54:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1667818453584, current_stage='None', description='', last_updated_timestamp=1667818453584, name='iris-classifier', run_id='0226e722f00a4413af31557fe1d6431a', run_link='', source='./artifacts_local/1/0226e722f00a4413af31557fe1d6431a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>